### Covid AirBnB Analysis

The Winter/Early Spring of 2020 was one that effected everyone around the entire world. The strike of Covid19, almost immediately crippled buisnesses and brought long term uncertanty. We immediately saw the impact in one particular city, New York City. As Corona virus girpped the country New York was hit by a wave (add facts here). As it was preading people began looking to leave the city to protect their health and their families. (potential additional analysis? - traffic flow out of the city or home buys/ sells etc). 

With the shutdown and exodus from NYC AirBnB saw a significant decrease in the use of its service. Here we will analyze data from www.insideairbnb.com where thought they are not affiliated with Airbnb the company says, "data utilizes public information compiled from the Airbnb web-site". 
 
**Procedure**
    1. We will begin by a quick observation of our data
    2. Clean the data, removing any missing data
    3. Feature Engineering of categorical features
    4. Feature Selection 
    

In [1]:
# loading basic libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
%matplotlib inline


In [2]:
# uploading data downloaded from insidedairbnb.com
listings = pd.read_csv('data/listings.csv')
calendar = pd.read_csv('data/calendar.csv')
reviews = pd.read_csv('data/reviews.csv')

data = [listings, calendar, reviews]  # creating list of our data frames 

In [3]:
def obs_data(dfs): # function for quick observation of our data
    for df in dfs:
        print('\n-----------------------Loading Next DF----------------------------\n')
        print('The shape of the dataframe is\n', df.shape, '\n')
        null_df = df[df.isna()]  # creating a df of columns records with null values
        null_cols = df.columns[df.isna().any()]  # creating a df of the column names to that are missing data 
        print('Names of columns with missing values')
        print(null_cols.tolist(), '\n')

        num_cols_mising = null_cols.nunique() # counting unique rows
        print('Number of Missing Columns')
        print(num_cols_mising)

        # percent of missing columns
        tot_cols = df.columns.nunique()
        print('\nPercent of columns missing records (%)')
        print(np.round((1-((tot_cols-num_cols_mising)/tot_cols))*100))

        # Missing Rows 
        num_missing_values = df.isna().sum()
#         print(num_missing_values.values)
        percent_missing_values = np.round(((1-(df.shape[0]-num_missing_values)/df.shape[0])*100),2)
#         # print(df.shape[0])
#         print(percent_missing_values)
        print('\nColumn','|' 'Percent of Missing Values (%)')
        print('----------------------------------- \n')
        for col, value in percent_missing_values.items():
            if value != 0:  # only printing columns and perentages that are > 0
                print((col),'|',(value))
            else:
                print
    print('\n...')
    print('...')
    print('...\n')
    print('DONE!') 

# running our list of dfs 
obs_data(data)



-----------------------Loading Next DF----------------------------

The shape of the dataframe is
 (46527, 74) 

Names of columns with missing values
['name', 'description', 'neighborhood_overview', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'license', 'reviews_per_mont

We can see some interesting insights from the intial analysis of this data. 
- Our calendar dataset, is the largest almost 17 million records, and 7 columns.
    - Out of 17 million records there were only 143 missing in two columns (minimum and maximum nights)
- Reviews is the second larges with about 1 million records and 6 fields.
    - Reviews had an insigificant 9 missing calues for reviewer name
- Finally we our left with our main table, our listings. With 46527 listings and 74 fields associated.
    - The listing data had the most missing values with 55% of the columns of the are missing records, with many missing 100% of the field.
    
 From this quick glance we observe various fields, are missing a wide range of records, and can implement different stratigies to handle those values. We will remove entire columns with more than 30% of the records missing, and remove the rows where the columns are missing minimal records. 

In [4]:
# Now we can write an equation to clean the data for us

def clean_data(dfs):  # cleans data based on percent (30%) missing from total values
    for df in dfs:
        df.dropna(thresh=(.3*df.shape[0]), axis=1, inplace=True) # dropping entire columns where the % of null values is greater than 30
        df.dropna(axis=0, inplace=True)  # drops the rows of all the other columns
        
        # double checking we've found all missing values
        empty_check = len((df.columns[df.isna().any()]).tolist())  # checking for empty columns converting to list and then counting the length
        if empty_check == 0:
            print (df.isna().any())
            print('No null values')
        else:
            print(df.isna().sum())
            print('We missed something')
        
# running again on our list of dfs        
clean_data(data)

id                                              False
listing_url                                     False
scrape_id                                       False
last_scraped                                    False
name                                            False
                                                ...  
calculated_host_listings_count                  False
calculated_host_listings_count_entire_homes     False
calculated_host_listings_count_private_rooms    False
calculated_host_listings_count_shared_rooms     False
reviews_per_month                               False
Length: 71, dtype: bool
No null values
listing_id        False
date              False
available         False
price             False
adjusted_price    False
minimum_nights    False
maximum_nights    False
dtype: bool
No null values
listing_id       False
id               False
date             False
reviewer_id      False
reviewer_name    False
comments         False
dtype: bool
No null values


We have no missing values!

In [5]:
# # Here we can save either as new file or overwrite (large files so we will pass for now)
# listings.to_csv('data/cleaned_listings.csv')
# calendar.to_csv('data/cleaned_calendar.csv')
# reviews.to_csv('data/cleaned_reviews.csv')

## Listing Information
From inital analysis we can devide our listing data it 3 segments that might factor into the rate of: rentals of the home
    1. Information about the host
    2. Neighborhood and Property information, and Price
    3. Retail listing scores

We'll break our listing information into two groups (we can always merge later):
1. Hosting information compared to listing scores and price
2. Neighbourhood and property information compared to listing scores and price

In [6]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'has_availability', 'avail

In [7]:
# selecting columns we are interested in 
host_raiting = listings[['id', 'host_name', 'host_since', 'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_listings_count',
       'host_total_listings_count','host_has_profile_pic', 'host_identity_verified', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable' ]]

host_raiting_cols = host_raiting.columns
# renaming columns using string matching
host_raiting.columns = [col.replace('host_','').replace('_scores_', '_') for col in host_raiting_cols] 

In [8]:
# working with categorical features
host_raiting_obj = host_raiting.select_dtypes(include ='object')
print(host_raiting_obj.columns.values)

['name' 'since' 'response_time' 'response_rate' 'acceptance_rate'
 'is_superhost' 'has_profile_pic' 'identity_verified' 'first_review'
 'last_review' 'instant_bookable']


Here when we look at our fields that our ojects we see something unexpected that happend when uploading our data:
   * Category "since" is a date - has to be converted.
   * "response_rate", and "acceptance_rate" read as strings - must be converted to float
   * String categorical categoriees that can be encoded using **Sklearn** label encoder

In [9]:
# manual featuring engineering individual variables
import datetime as dt
now = dt.datetime.now().date()  # coverts date times to just date with no time
host_raiting['since'] = pd.to_datetime(host_raiting['since'])
host_raiting['first_review'] = pd.to_datetime(host_raiting['first_review'])
host_raiting['last_review'] = pd.to_datetime(host_raiting['last_review'])

host_raiting['year'] = host_raiting['since'].dt.year
host_raiting['month'] = host_raiting['since'].dt.month

host_raiting['total_days_rental'] = (now - host_raiting['since'].dt.date) # time difference must confirm datetime to date to match
host_raiting['days_since_first_rev'] = (now - host_raiting['first_review'].dt.date)
host_raiting['days_since_last_rev'] = (now - host_raiting['last_review'].dt.date)

# feature engineering rates 
def p2f(x):  # function that strips the % sign and converts to decimal
    return float(x.strip('%'))/100

host_raiting['response_rate'] = host_raiting['response_rate'].apply(lambda x: p2f(x))
host_raiting['acceptance_rate'] = host_raiting['acceptance_rate'].apply(lambda x: p2f(x))

# feat engieering of true and falses
def tof(x):
    if x == 't':
        return 1
    elif x == 'f':
        return 0
    
host_raiting['is_superhost'] = host_raiting['is_superhost'].apply(lambda x: tof(x))
host_raiting['has_profile_pic'] = host_raiting['has_profile_pic'].apply(lambda x: tof(x))
host_raiting['identity_verified'] = host_raiting['identity_verified'].apply(lambda x: tof(x))
host_raiting['instant_bookable'] = host_raiting['instant_bookable'].apply(lambda x: tof(x))

# one hot encoding using sklearn 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

le = LabelEncoder()
ohe = OneHotEncoder()

# label encoding response time
encoded = le.fit_transform(host_raiting['response_time'])
le_encoded = encoded.reshape(len(encoded), 1)
host_raiting['response_time_encoded'] = le_encoded 
# print(le.inverse_transform(le_encoded))

# label encoding names
le2 = LabelEncoder()
names_encoded = le2.fit_transform(host_raiting['name'])
le_names_encoded = names_encoded.reshape(len(names_encoded), 1)
host_raiting['names_encoded'] = le_names_encoded 
# print(le2.inverse_transform(le_names_encoded))

# for uniq in host_raiting['response_time'].unique():
#     if uniq == 'within a few hours':
#         host_raiting[uniq] = 1
#     elif uniq == 'within an hour':
#         host_raiting[uniq] = 1
#     elif uniq == 'within a day':
#         host_raiting[uniq] = 1
#     elif uniq == 'a few days or more':
#         host_raiting[uniq] = 1
#     else:
#         host_raiting[uniq] = 0
display(host_raiting.head(50))
print(host_raiting.select_dtypes(include='object').columns)

<ipython-input-9-ece18af42ca9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_raiting['since'] = pd.to_datetime(host_raiting['since'])
<ipython-input-9-ece18af42ca9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_raiting['first_review'] = pd.to_datetime(host_raiting['first_review'])
<ipython-input-9-ece18af42ca9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,id,name,since,response_time,response_rate,acceptance_rate,is_superhost,listings_count,total_listings_count,has_profile_pic,...,review_location,review_value,instant_bookable,year,month,total_days_rental,days_since_first_rev,days_since_last_rev,response_time_encoded,names_encoded
1,3831,LisaRoxanne,2008-12-07,within a few hours,0.90,0.96,0,1.0,1.0,1,...,9.0,9.0,0,2008,12,4314 days,2191 days,59 days,2,1407
4,5178,Shunichi,2009-03-03,within a few hours,1.00,1.00,0,1.0,1.0,1,...,10.0,9.0,0,2009,3,4228 days,4164 days,198 days,2,2118
6,5238,Ben,2009-02-07,within a few hours,1.00,0.18,1,4.0,4.0,1,...,9.0,9.0,0,2009,2,4252 days,4046 days,428 days,2,260
8,5803,Laurie,2009-03-10,within a few hours,1.00,0.98,1,3.0,3.0,1,...,10.0,10.0,0,2009,3,4221 days,4177 days,195 days,2,1350
10,6090,Alina,2009-04-01,within a few hours,1.00,1.00,0,2.0,2.0,1,...,10.0,9.0,1,2009,4,4199 days,4055 days,66 days,2,87
12,6990,Cyn,2009-05-12,within an hour,1.00,0.93,1,3.0,3.0,1,...,10.0,10.0,1,2009,5,4158 days,3989 days,295 days,3,483
13,7097,Jane,2009-05-17,within an hour,1.00,1.00,1,1.0,1.0,1,...,10.0,9.0,1,2009,5,4153 days,3909 days,71 days,3,1028
22,10962,Laurie,2009-03-10,within a few hours,1.00,0.98,1,3.0,3.0,1,...,10.0,10.0,0,2009,3,4221 days,4020 days,57 days,2,1350
24,12048,Ben,2009-02-07,within a few hours,1.00,0.18,1,4.0,4.0,1,...,10.0,10.0,0,2009,2,4252 days,3988 days,321 days,2,260
25,12192,Edward,2009-10-19,within a day,0.60,0.73,0,2.0,2.0,1,...,10.0,10.0,0,2009,10,3998 days,3990 days,196 days,1,646


Index(['name', 'response_time'], dtype='object')


In [10]:
# feature engineering neighbourhood and property info by encoding - the easy way
demog_raiting = listings[['id', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability', 'availability_30',
       'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable']]
                         
demog_le = LabelEncoder()
demog_encoded = demog_raiting.apply(demog_le.fit_transform)  # simple way to label encode over multiple cols must be aware of 
# can find classes using the inverse_transfrom as before

demog_encoded.head()

,id,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
1,0,13,35,1,1576,1926,8,0,2,2,...,596,773,31,5,7,6,6,6,6,0
4,1,54,86,2,4205,1015,19,2,1,2,...,1,635,26,6,5,6,6,7,6,0
6,2,54,29,2,2641,768,3,0,1,2,...,3,408,36,7,7,7,7,6,6,0
8,3,13,162,1,884,893,31,2,1,5,...,0,638,36,7,8,7,7,7,7,0
10,4,54,186,2,3330,246,3,0,1,2,...,2,766,39,7,7,6,7,7,6,1


In [11]:
# saving listings partitons
host_raiting.to_csv('data/host_info.csv')
demog_encoded.to_csv('data/rental_info.csv')

## Calendar Information
Here we are given calander for the listings. Will be able to aggregate and find out how many nights out of year a listing is reseverd and so on.

In the Calendar table, we have 7 columns, no missing data, howevver see problems with our data types:
* Date is an object field - should be date
* Price and adjusted price has a string in it and classified as object - should be float
* Available is categorical however should be numerically encoded


In [12]:
print(calendar.info())
calendar.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16979833 entries, 0 to 16979975
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.0+ GB
None


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,45910,2020-08-17,t,$325.00,$325.00,4.0,730.0
1,18127,2020-08-16,f,$170.00,$170.00,5.0,30.0
2,18127,2020-08-17,f,$170.00,$170.00,5.0,30.0
3,18127,2020-08-18,f,$170.00,$170.00,5.0,30.0
4,18127,2020-08-19,t,$170.00,$170.00,5.0,30.0


In [13]:
# feature engineering calendar table
# working with dates
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['year'] = calendar['date'].dt.year
calendar['month'] = calendar['date'].dt.month

# removing string variable in price using regular expressions
# import re
# calendar['price'] = re.sub('calendar['price'].str.lstrip('$'))
# calendar['adjusted_price'] = re.sub('calendar['adjusted_price'].str.lstrip('$'))

calendar[['price','adjusted_price']] = calendar[['price','adjusted_price']].replace({'\$': '', ',': ''}, regex=True).astype(float)

# we can use the function we defined earlier
calendar['available'] = calendar['available'].apply(lambda x: tof(x))
calendar.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16979833 entries, 0 to 16979975
Data columns (total 9 columns):
 #   Column          Dtype         
---  ------          -----         
 0   listing_id      int64         
 1   date            datetime64[ns]
 2   available       int64         
 3   price           float64       
 4   adjusted_price  float64       
 5   minimum_nights  float64       
 6   maximum_nights  float64       
 7   year            int64         
 8   month           int64         
dtypes: datetime64[ns](1), float64(4), int64(4)
memory usage: 1.3 GB


We have successfuly created some features and also corrected the datatypes of our exsiting features. Now that we have completed these two sections we can merege our calander with our listings information or or segmented listing info and such. We can save it as a new file or overwrite (it is large).


## Reviews

The reviews table is a bit different in composition than others. It is composed of 10.5 million reviews of the listings on Airbnb. Many of the listings have a one to many relationship with the reviews, same goes with homeowners and such for futer analysis.

For the reviews table, in this section, we will only change the data types of the fields, and do some feature engineering. For future work and in future notebooks we will apply Natural Language Processing (NLP) to analyze the sentiment of reviews as well as the descriptions in the listing table for correlations.

In [14]:
print(reviews.info())
reviews.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058597 entries, 0 to 1059287
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1058597 non-null  int64 
 1   id             1058597 non-null  int64 
 2   date           1058597 non-null  object
 3   reviewer_id    1058597 non-null  int64 
 4   reviewer_name  1058597 non-null  object
 5   comments       1058597 non-null  object
dtypes: int64(3), object(3)
memory usage: 56.5+ MB
None


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2595,17857,2009-11-21,50679,Jean,Notre séjour de trois nuits.\r\nNous avons app...
1,2595,19176,2009-12-05,53267,Cate,Great experience.
2,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...
3,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en..."
4,2595,46312,2010-05-25,117113,Alicia,We had a wonderful stay at Jennifer's charming...


We are presented with 6 columns, we see the same issue with date as the other dataframes, other than that we will drop id (it has to do with the individual review), and also reviewer name because reviewer_id is more useful for analysis. 

In [15]:
reviews = pd.read_csv('data/reviews.csv')

In [16]:
# feature engineering
# dropping unessecary columns
# reviews.drop('id', axis=1, inplace=True)
reviews.drop('reviewer_name', axis = 1, inplace=True)

# feature engieering dates
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['year'] = reviews['date'].dt.year
reviews['month'] = reviews['date'].dt.month

# rearanging the columns
reviews = reviews[['listing_id', 'date', 'year', 'month', 'comments']]
reviews.head()

,listing_id,date,year,month,comments
0,2595,2009-11-21,2009,11,Notre séjour de trois nuits.\r\nNous avons app...
1,2595,2009-12-05,2009,12,Great experience.
2,2595,2009-12-10,2009,12,I've stayed with my friend at the Midtown Cast...
3,2595,2010-04-09,2010,4,"We've been staying here for about 9 nights, en..."
4,2595,2010-05-25,2010,5,We had a wonderful stay at Jennifer's charming...
